In [1]:
import os
from mistralai import Mistral

In [2]:
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-small-latest"

client = Mistral(api_key=api_key)

In [3]:
def call_strong(messages: list) -> str:
    chat_response = client.chat.complete(
        model= "mistral-large-latest",
        messages = messages
    )
    return chat_response.choices[0].message.content

In [4]:
def call_mini(messages: list) -> str:
    chat_response = client.chat.complete(
        model= "mistral-small-latest",
        messages = messages
    )
    return chat_response.choices[0].message.content

### Prereq

In [5]:
from reasonable import DefaultReasoningAgent

In [6]:
agent = DefaultReasoningAgent(
    main_function=call_mini,
    thoughts_function=call_mini,
    max_steps=10,
    timeout=2,
    verbose=True
)

ReasoningAgent initialized. Verbose mode is on.


In [8]:
import re
import time

def evaluate_llm_response(question, llm_answer_func, evaluator_func):
    question_text = question["question"]
    correct_solution = question["solution"]
    correct_answer = question["answer"]

    time.sleep(5)
    llm_answer = llm_answer_func([{"role": "user", "content": question_text}])

    evaluation_prompt = (
        f"Question: {question_text}\n"
        f"Correct solution: {correct_solution}\n"
        f"Correct answer: {correct_answer}\n"
        f"Answer by student: {llm_answer}\n"
        "Please evaluate the correctness of the student's answer above."
        "Evaluate the student's answer in <evaluation> tag and provide a verdict ('correct' or 'incorrect') in <verdict> tag and score (from 0 to 10) in <score> tag."
        "Respond in this format:"
        "<evaluation>"
        "..."
        "</evaluation>"
        "<verdict>"
        "..."
        "</verdict>"
        "<score>"
        "..."
        "</score>"
    )
    
    time.sleep(5)
    

    evaluation_result = evaluator_func([{"role": "user", "content": evaluation_prompt}])

    # extract score
    pattern = r'<score>(.*?)</score>'
    score = re.search(pattern, evaluation_result, re.DOTALL)
    if score:
        score = int(score.group(1).strip())
    else:
        score = 0

    # extract verdict
    pattern = r'<verdict>(.*?)</verdict>'
    verdict = re.search(pattern, evaluation_result, re.DOTALL)
    if verdict:
        verdict = verdict.group(1).strip()
        if "incorrect" in verdict:
            verdict = 0
        else:
            verdict = 1
    else:
        verdict = 0

    return {"question": question_text, "correct_answer": correct_answer, "llm_answer": llm_answer, "score": score, "verdict": verdict}

### Dataset

In [9]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceH4/MATH-500")

In [10]:
import random

In [11]:
MATH50_ds = []
for example in ds["test"]:
    MATH50_ds.append({
        "question": example["problem"],
        "solution": example["solution"],
        "answer": example["answer"]
    })

In [12]:
random.shuffle(MATH50_ds)

In [13]:
MATH50_ds = MATH50_ds[:50]

In [14]:
MATH50_ds

[{'question': 'Evaluate $\\log_264$.',
  'solution': 'We have $2^6=64$, so $\\log_2 64 = \\boxed{6}$.',
  'answer': '6'},
 {'question': 'What is $\\left(4\\dfrac{5}{8}\\right)^{55} \\cdot \\left(\\dfrac{8}{37}\\right)^{55}$?',
  'solution': 'First we convert $4\\dfrac{5}{8}$ into an improper fraction: \\[4\\dfrac{5}{8} = 4 + \\dfrac{5}{8} = \\dfrac{32}{8} + \\dfrac{5}{8} = \\dfrac{37}{8}.\\]We discover that $4\\dfrac{5}{8}$ and $\\dfrac{8}{37}$ are in fact reciprocals of each other. Using the fact that $(ab)^n = a^nb^n$, we get our answer: \\[\n\\left(4\\dfrac{5}{8}\\right)^{55} \\cdot \\left(\\dfrac{8}{37}\\right)^{55} = \\left(4\\dfrac{5}{8} \\cdot \\dfrac{8}{37}\\right)^{55} = 1^{55} = \\boxed{1}.\\]',
  'answer': '1'},
 {'question': 'Determine the modulo 4 remainder of the following sum: $$ 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 11 + 12. $$',
  'solution': 'Grouping residues helps make some series computations easier:  \\begin{align*}\n1 + 2 + 3 + 0 + 1 + 2& + 3 + 0 + 1 + 2 + 3 +

### Evaluating base model

In [9]:
from tqdm import tqdm

In [36]:
base_model_test_info = {
    "total": 0,
    "score": 0,
    "answers" : []
}

max_total = 50
max_score = 50 * 10

for question in tqdm(MATH50_ds):
    foo = evaluate_llm_response(question, call_mini, call_mini)
    base_model_test_info["total"] += foo["verdict"]
    base_model_test_info["score"] += foo["score"]
    base_model_test_info["answers"].append(foo)

    print(f"Current accuracy: {base_model_test_info['total']/len(base_model_test_info['answers'])}/1.0")
    print(f"Current score: {base_model_test_info['score']}/{10*len(base_model_test_info['answers'])}")
    time.sleep(1)

  0%|          | 0/50 [00:00<?, ?it/s]

Current accuracy: 1.0/1.0
Current score: 10/10


  2%|▏         | 1/50 [00:13<11:04, 13.56s/it]

Current accuracy: 1.0/1.0
Current score: 20/20


  4%|▍         | 2/50 [00:27<11:02, 13.81s/it]

...

 96%|█████████▌| 48/50 [12:30<00:29, 14.73s/it]

Current accuracy: 0.6938775510204082/1.0
Current score: 402/490


 98%|█████████▊| 49/50 [12:45<00:14, 14.94s/it]

Current accuracy: 0.68/1.0
Current score: 402/500


100%|██████████| 50/50 [13:01<00:00, 15.63s/it]


In [37]:
import json

with open("mistral_base_model_test_info.json", 'w', encoding='utf-8') as json_file:
    json.dump(base_model_test_info, json_file, ensure_ascii=False, indent=4)

In [38]:
with open("mistral_MATH50_ds.json", 'w', encoding='utf-8') as json_file:
    json.dump(MATH50_ds, json_file, ensure_ascii=False, indent=4)

### Open json

In [10]:
import json

with open("mistral_MATH50_ds.json", 'r', encoding='utf-8') as json_file:
    MATH50_ds = json.loads(json_file.read())

### Eval reasoner

In [11]:
def reasonable_wrapper(messages):
    message = messages[-1]
    response = agent.reason(message["content"])
    return response['answer']

In [ ]:
reasoning_model_test_info = {
    "total": 0,
    "score": 0,
    "answers" : []
}

max_total = 50
max_score = 50 * 10

for question in tqdm(MATH50_ds):
    try:
        foo = evaluate_llm_response(question, reasonable_wrapper, reasonable_wrapper)
        reasoning_model_test_info["total"] += foo["verdict"]
        reasoning_model_test_info["score"] += foo["score"]
        reasoning_model_test_info["answers"].append(foo)
    except:
        pass

    print(f"Current accuracy: {reasoning_model_test_info['total']/len(reasoning_model_test_info['answers'])}/1.0")
    print(f"Current score: {reasoning_model_test_info['score']}/{10*len(reasoning_model_test_info['answers'])}")

  0%|          | 0/50 [00:00<?, ?it/s]

Parsed from assistant:
  reasoning: This problem requires us to evaluate the logarithm base 2 of 64. To solve this, we need to find the exponent to which the base 2 must be raised to yield 64. Let's start by considering the properties of logarithms and exponential functions. First, I will use my knowledge of logarithms and exponentials to estimate the answer.
  next_action: continue
Parsed from assistant:
  reasoning: To evaluate $\log_264$, we need to find the exponent to which the base 2 must be raised to produce 64. This is a problem that can be solved by applying the definition of logarithms. Let's start by considering the prime factorization of 64 to determine the power of 2 that gives 64. The prime factorization of 64 is $2^6 = 64$.

We know that $2^6 = 64$, so $\log_264 = 6$. We also know that $2^5 = 32$ and $2^7 = 128$. Therefore, $6$ is the correct exponent.

To confirm this, we can use another method. We can use the change of base formula which states that $\log_b a = \frac{\